# Phase 3 project

## Covid Dataset

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import joblib
import xgboost as xgb

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score
from sklearn.metrics import roc_curve, auc, plot_roc_curve, roc_auc_score
from sklearn.metrics import confusion_matrix, plot_confusion_matrix, classification_report

from sklearn.linear_model import LogisticRegression
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier

from colorama import Fore
from colorama import Style

import warnings
warnings.filterwarnings('ignore')

KeyboardInterrupt: 

In [ ]:
df_covid=pd.read_csv('Covid Data.csv')

In [ ]:
df_covid

### Description of Dataset

About Dataset
Context
Coronavirus disease (COVID-19) is an infectious disease caused by a newly discovered coronavirus. Most people infected with COVID-19 virus will experience mild to moderate respiratory illness and recover without requiring special treatment. Older people, and those with underlying medical problems like cardiovascular disease, diabetes, chronic respiratory disease, and cancer are more likely to develop serious illness.
During the entire course of the pandemic, one of the main problems that healthcare providers have faced is the shortage of medical resources and a proper plan to efficiently distribute them. In these tough times, being able to predict what kind of resource an individual might require at the time of being tested positive or even before that will be of immense help to the authorities as they would be able to procure and arrange for the resources necessary to save the life of that patient.

The main goal of this project is to build a machine learning model that, given a Covid-19 patient's current symptom, status, and medical history, will predict whether the patient is in high risk or not.

### content
The dataset was provided by the Mexican government (link). This dataset contains an enormous number of anonymized patient-related information including pre-conditions. The raw dataset consists of 21 unique features and 1,048,576 unique patients. In the Boolean features, 1 means "yes" and 2 means "no". values as 97 and 99 are missing data.

* sex: 1 for female and 2 for male.
* age: of the patient.
* classification: covid test findings. Values 1-3 mean that the patient was diagnosed with covid in different degrees. 4 or higher means that the patient is not a carrier of covid or that the test is inconclusive.
* patient type: type of care the patient received in the unit. 1 for returned home and 2 for hospitalization.
* pneumonia: whether the patient already have air sacs inflammation or not.
* pregnancy: whether the patient is pregnant or not.
* diabetes: whether the patient has diabetes or not.
* copd: Indicates whether the patient has Chronic obstructive pulmonary disease or not.
* asthma: whether the patient has asthma or not.
* inmsupr: whether the patient is immunosuppressed or not.
* hypertension: whether the patient has hypertension or not.
* cardiovascular: whether the patient has heart or blood vessels related disease.
* renal chronic: whether the patient has chronic renal disease or not.
* other disease: whether the patient has other disease or not.
* obesity: whether the patient is obese or not.
* tobacco: whether the patient is a tobacco user.
* usmr: Indicates whether the patient treated medical units of the first, second or third level.
* medical unit: type of institution of the National Health System that provided the care.
* intubed: whether the patient was connected to the ventilator.
* icu: Indicates whether the patient had been admitted to an Intensive Care Unit.
* date died: If the patient died indicate the date of death, and 9999-99-99 otherwise.

In [ ]:
df_covid.columns

The column 'CLASSIFICATION_FINAL' tells us which patients tested positive for Covid and which ones didn't.
Values 1-3 mean that the patient was diagnosed with covid in different degrees. 4 or higher means that the patient is not a carrier of covid or that the test is inconclusive. 
So since we are interested only in data regarding covid positive patients, we will filter out the rows that have a value greater than 3.

In [ ]:
df_covid['CLASIFFICATION_FINAL'].value_counts(normalize=True)

In [ ]:
df_covid.drop(df_covid.loc[df_covid['CLASIFFICATION_FINAL']>3].index, inplace=True)

In [ ]:
df_covid['CLASIFFICATION_FINAL'].value_counts(normalize=True)

In [ ]:
df_covid.shape

We still have a good amount of entries left.
And now that we made sure our dataframe contains only covid positive patients, I need to decide which one should be the target variable. <br>The possible ones are:'patient type', 'usmr', 'medical unit', 'icu', 'date died'

In [ ]:
### ADD MARKDOWN to describe the columns

In [ ]:
df_covid['USMER'].value_counts(normalize=True)

In [ ]:
df_covid['PATIENT_TYPE'].value_counts(normalize=True)

In [ ]:
### This could be good but it is imbalanced. But maybe even like this it is ok.

In [ ]:
df_covid['MEDICAL_UNIT'].value_counts(normalize=True)

In [ ]:
# Not very informative

In [ ]:
df_covid['ICU'].value_counts(normalize=True)

In [ ]:
# Mostly missing values

In [ ]:
df_covid['DATE_DIED'].value_counts(normalize=True)

In [ ]:
# Mostly missing values

Stays standing 'classification', 'patient type'

To make the decision between these two variables we did some research to gain some domain knowledge and spoke directly to some first responders during the covid 19 pandemic, that suggested that the information about whether or not a patient needed to be hospitalized was more valuable than the results of the covid test.
This is the case since some patients might have tested positive for covid 19, but because of mild symptoms and overall good health were sent back home to be treated, and what actually really put a strain on health structures was the number of people in need to be hospitalized. Because of this we are going to use 'patient type' as our target for this study.
We are also going to drop all the other variables related to hospitalization since those contain knowledge about the hospitalization of the patient, while what we are trying to predict is the amout of people that would need to be hospitalized, based on previous knowledge so this is not a type of information we would have, if we wanted to use the model again on other data.

So target should be patient type and all the other hospital related variables should be dropped.

In [ ]:
df=df_covid.drop(['USMER', 'MEDICAL_UNIT', 'INTUBED', 'CLASIFFICATION_FINAL', 'ICU', 'DATE_DIED'], axis=1)

In [ ]:
df

The missing values are catalogued as 97 or 99. So I have to look for these values to understand how many null values.

Will need to find a way to replace the missing values for pregnant.

In [ ]:
for i in df.columns:
    print(df[i].value_counts(normalize=True))
    print("\n")

97, 98 and 99 represent missing values for all the columns except 'AGE'.
<br>For some categories the missing values are so little that it's worth just dropping them. <br>For the two categories with the most missing values, 'PREGNANT' and 'PNEUMONIA' we will proceed to impute them.

We will temporarily remove the 'age' column as in this one 97, 98 and 99 are actually real values and not missing values.
<br> We will work on this column first to remove the outliers, then we will drop it from the dataset and reeinsert it once we dealt with the missing values for the other columns.

In [ ]:
age=df['AGE']

In [ ]:
plt.hist(age);

Even if we cannot be certain about this, since there is no information about this on the dataset, we can safely assume that values for age that are above 110 are probably typos or outliers. It might also be the way that it was inputed when the value of 'age' for that patient was missing: since there are no NaN values and yet, 97 98 and 99 which were used in the other columns to indicate missing values, clearly cannot be used in this one.
So we will remove the columns that have the age value>110.

In [ ]:
df.drop(df.loc[df['AGE']>110].index, inplace=True)

In [ ]:
df['AGE'].value_counts()

Now that we have taken care of the outliers for this column in the Dataframe we can remove it to add it again once we finish the rest of the data processing.

In [ ]:
df.drop('AGE', axis=1, inplace=True)

In [ ]:
for i in df.columns:
    df[i]=df[i].replace([97,98,99], np.NaN)

In [ ]:
df.isna().sum()

In [ ]:
"""I could replace the missing values using percentages OR I could do it using KNN OR I could do it direclty in the decision tree/random forest.
For now let me replace them in this easier percentage way, then if I have time later I will try the other two ways."""

As we saw before most missing values are in the column "PREGNANT".
<br>We want to replace them but not just with the mode, but keeping the same percentage of values from the original data.
<br>We will look at how the percentage is distributed, filtering out for now the null values.

In [ ]:
preg=pd.DataFrame()
preg=df['PREGNANT']
preg.drop(preg.loc[preg>3].index, inplace=True)

In [ ]:
preg.value_counts(normalize=True)

In [ ]:
df['PREGNANT'].fillna(np.random.choice([1, 2], 
                                    p=[0.02,0.98]), inplace=True)

In [ ]:
df['PREGNANT'].value_counts()

In [ ]:
df['PREGNANT']


In [ ]:
df.isna().sum()

In [ ]:
df=pd.concat([df, age], axis=1)

In [ ]:
df.dropna(inplace=True)

In [ ]:
df.isna().sum()

In [ ]:
df

In [ ]:
###Correlation Matrix see if there's a better place for it.

In [ ]:
fig, ax = plt.subplots(figsize=(15, 12))
sns.heatmap(df.corr(), center=0, ax=ax, annot=True, mask=np.triu(np.ones_like(df.corr(), dtype=bool)), cmap="Blues");

Now to follow the usual convention of one hot encoding I want to change the values 2 which means no into 0.
<br>This is true for all the columns except our target, 'PATIENT_TYPE' where 1 means they returned home and 2 means they were hospitalized. I will do this separately, and then assign the column to y.
For X I will do the necessary changes to df and then concat with 'AGE' that I dropped before.
<br>For sex it is going to be 1 for female and 0 for male.

In [ ]:
y=df['PATIENT_TYPE']

In [ ]:
#I have to redifine age again because the old onw has some null values that I dropped.
age=df['AGE']

In [ ]:
df.drop(['PATIENT_TYPE','AGE'], axis=1, inplace=True)

In [ ]:
df=df.replace(2, 0)

In [ ]:
df

Changing to zero and 1 for the target:

In [ ]:
y=y.replace(1,0)
y=y.replace(2, 1)
y

In [ ]:
y.value_counts()

In [ ]:
X=pd.concat([df,age], axis=1)

In [ ]:
X

At this point I took care of the missing values and I don't need to do one hot encoding.
<br>I have my X and y, I can divide in train and test and start trying some models.
<br>But if I want to use KNN first (and I do) I have to scale first, specifically age needs to be scaled.

In [ ]:
"""Another thing I could try later to improve the model is instead of scaling the age to divide it into bins. Like 10 years for each bin and one hot encode that."""

### Splitting into train and test set

In [ ]:
# Split the data
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=42)

In [ ]:
# Instantiate StandardScaler
scaler = MinMaxScaler()
# Transform the training and test sets
scaled_data_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Convert into a DataFrame
X_train = pd.DataFrame(scaled_data_train, columns=X.columns)
X_train.head()

### Preliminary model with Logistic Regression:

A simple model that we can try is Logistic Regression.<br> I works similarly to Linear Regression but it can make predictions on categorical data, splitting continuous intervals into beams.

In [ ]:
logreg = LogisticRegression(max_iter=500, random_state=19)
logreg.fit(X_train,y_train)

In [ ]:
pred_train_log = logreg.predict(X_train)

We will take a chance here to define a few functions that will help us preview our results.

In [ ]:
# defined so that I can use it for test or train
def print_metrics(labels, preds):
    print("Precision Score: {}".format(precision_score(labels, preds)))
    print("Recall Score: {}".format(recall_score(labels, preds)))
    print("Accuracy Score: {}".format(accuracy_score(labels, preds)))
    print("F1 Score: {}".format(f1_score(labels, preds)))

In [ ]:
def save_metrics(model, labels, preds):
    results=[]
    results.append(model)
    results.append(['precision', (precision_score(labels, preds))])
    results.append(['recall', (recall_score(labels, preds))])
    results.append(['accuracy', (accuracy_score(labels, preds))])
    results.append(['f1', (f1_score(labels, preds))])
    yhat = model.predict_proba(X_train)
    yhat = yhat[:, 1]
    fpr, tpr, thresholds = roc_curve(y_train, yhat)
    results.append(['auc', (auc(fpr,tpr))])
    return results

In [ ]:
def plot_matrix(labels, preds, cmap=None):
    cm = confusion_matrix(labels, preds)
    # To Normalize
    cmn = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    fig, ax = plt.subplots(figsize=(6,5))
    sns.heatmap(cmn, annot=True, fmt='.2f', cmap="OrRd")
    plt.ylabel('Actual')
    plt.xlabel('Predicted')
    plt.show(block=False)

In [ ]:
def model_results(model):
    with plt.style.context('seaborn-talk'):
        print(f'{Fore.RED}Results for the model {model}{Style.RESET_ALL}')
        print(f'{Fore.BLUE}TRAIN{Style.RESET_ALL}')
        y_preds_train=model.predict(X_train)
        print_metrics(y_train, y_preds_train)
        roc_score_train = roc_auc_score(y_train, model.predict_proba(X_train)[:, 1])
        print(f"Train Roc_Auc Score: {roc_score_train :.2%}")
        print('Confusion Matrix Train:')
        plot_matrix(y_train, y_preds_train)
        print(f'{Fore.GREEN}TEST{Style.RESET_ALL}')
        y_preds_test=model.predict(X_test)
        print_metrics(y_test, y_preds_test)
        roc_score_test = roc_auc_score(y_test, model.predict_proba(X_test)[:, 1])
        print(f"Test Roc_Auc Score: {roc_score_test :.2%}")
        print('Confusion Matrix Test:')
        plot_matrix(y_test, y_preds_test)
        plt.show() 

In [ ]:
def plot_roc(models, labels):
    colors=['blue', 'green', 'k', 'purple','orange', 'm', 'c', 'b', 'lime' ]
    lines=['dashed', 'dashdot', 'dotted', 'dashed', 'dashdot', 'dotted', 'dashed', 'dashdot', 'dotted']
    plt.figure(figsize=(10,8))
    for i, j, k in zip(models, lines, colors):
        yhat = i.predict_proba(X_train)
        yhat = yhat[:, 1]
        fpr, tpr, thresholds = roc_curve(y_train, yhat)
        plt.plot(fpr, tpr, label=i, alpha=0.6, linestyle=j, color=k, linewidth=3)
    plt.plot([0, 1], [0, 1], color='indigo', linestyle='--', label='No Skill')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver operating characteristic (ROC) Curve')
    plt.legend(loc='lower right')
    plt.show()


In [ ]:
def compare_roc(model):
    plt.figure(figsize=(10,8))
    yhat = i.predict_proba(X_train)
    yhat = yhat[:, 1]
    fpr, tpr, thresholds = roc_curve(y_train, yhat)
    plt.plot(fpr, tpr, label='train', alpha=0.6, linestyle='dashed', color='forestgreen', linewidth=3)
    yhat_test = i.predict_proba(X_test)
    yhat_test = yhat_test[:, 1]
    fpr, tpr, thresholds = roc_curve(y_test, yhat_test)
    plt.plot(fpr, tpr, label='test', alpha=0.6, linestyle='dashdot', color='firebrick', linewidth=3)
    plt.plot([0, 1], [0, 1], color='indigo', linestyle='--', label='No Skill')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver operating characteristic (ROC) Curve')
    plt.legend(loc='lower right')
    plt.show()

In [ ]:
"""# with Seaborn
def plot_roc(models, labels):
#    markers=[ '1','2','3','4','+','*', 'x']
    plt.figure(figsize=(12,8))
    for i in models:#, markers, colors):
        yhat = i.predict_proba(X_train)
        yhat = yhat[:, 1]
        fpr, tpr, thresholds = roc_curve(y_train, yhat)
        sns.lineplot(x=fpr, y=tpr, label=i, palette="flare", lw=0.5)#, marker=j, alpha=0.6, color=k)#linewidth=0.5,
    sns.lineplot([0, 1], [0, 1], color='navy', linestyle='--', label='No Skill')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver operating characteristic (ROC) Curve')
    plt.legend(loc='lower right')
    plt.show()"""

In [ ]:
print_metrics(y_train, pred_train_log)

In [ ]:
plot_matrix(y_train, pred_train_log)

In [ ]:
plot_roc([logreg], y_train)

In [ ]:
compare_roc(logreg)

In [ ]:
#resu_logreg=save_metrics(logreg, y_train, pred_train_log)

In [ ]:
#resu_logreg

### Decision tree
The Logistic Regression Model will be kept as our baseline model.
<br>We will try next with a Decision Tree, and see if it performs better.

In [ ]:
#Create the tree
DT1 = DecisionTreeClassifier(criterion='entropy', random_state=19)

In [ ]:
# fit the tree
DT1.fit(X_train, y_train)

In [ ]:
# Evaluate the tree
print("For the train with decision tree:\n")
preds_train_DT1 = DT1.predict(X_train)
model_results(DT1)

In [ ]:
plot_roc([logreg,DT1], y_train)

In [ ]:
compare_roc(DT1)

Another feature that we have with decision trees is that we can extract their feature importance.

In [ ]:
def plot_feature_importances(model):
    n_features = X_train.shape[1]
    plt.figure(figsize=(5,5))
    #not sure why when I add np.sort in front of model feature it changes
    plt.barh(range(n_features), (model.feature_importances_), align='center') 
    plt.yticks(np.arange(n_features), X_train.columns.values) 
    plt.xlabel('Feature importance')
    plt.ylabel('Feature')

In [ ]:
plot_feature_importances(DT1)

In [ ]:
#resu_DT1=save_metrics(DT1, y_train, preds_train_DT1)

In [ ]:
#resu_DT1

A few more information that we can retrieve about our tree:

In [ ]:
print(f'Decision Tree has {DT1.tree_.node_count} nodes with a maximum depth of {DT1.tree_.max_depth}.')
print(f'Model Accuracy for train data: {DT1.score(X_train, y_train)}')

This is not so bad as a result. But we have to ekep in mind that because of our class imbalance (80% of the patients returned home and 20% got hospitalized) a baseline model would have an accuracy of 80%.
<br><br>
Also - given our situation I would actually like to have a higher recall than accuracy. With a high recall it means I am getting more false positives because I am decreasing the thershold, but that is what I want most times in 'medical situations', because I would rather have a false positive than a false negative.

### Class Imbalance
<br> One thing we have not considered yet is the fact that the classes of our target (patients sent home vs. patients hospitalized) are not balanced. They are not present equally in our dataset but around 80% of the patients were sent home and only 20% were hospitalized, as we saw from the split of the data for that column.
<br> This means that a very basic model that simply predicts all the patients to be sent home would have an 80% accuracy, that's the baseline.
<br> We have a way to counteract this problem by balancing the classes with a decision tree and that is what we are going to do.

In [ ]:
DT_bal = DecisionTreeClassifier(criterion='entropy', class_weight='balanced', random_state=19)

In [ ]:
DT_bal.fit(X_train, y_train)

In [ ]:
preds_train_DTbal = DT_bal.predict(X_train)

In [ ]:
# Evaluate the tree
print("Results with Decision Tree considering class imbalance:\n")
model_results(DT_bal)

In [ ]:
plot_roc([logreg, DT1, DT_bal], y_train)

In [ ]:
compare_roc(DT_bal)

In [ ]:
plot_feature_importances(DT_bal)

As we can see from the confusion matrix we started to have more False Positives and the predictions of the majority class (0, people returned home) has decreased. This reflects also in the accuracy score and F1 score.
<br> But on the bright side the predictions for the minority class (1, hospitalized) have improved, with less false negatives and a recall that went from 0.63 to 0.77.
<br> This is a matter of choice for the stakeholder, which model to use. The Decision tree without class imbalance leads to an overall more precise model: less falses overall, negative and positive, more patients categorized correctly.
<br> On the other hand the model that accounts for class imbalance makes more mistakes, has lower accurcay precision and F1 score, but it has overall less false negatives.
<br>Given the specific problem we would recommend staying on the safer side, trying to minimize the false negatives then the false positives, because it would be less of a loss to be prepared for one extra patient that doesn't ultimately need to be hospitalized, rather then not being ready with the equippment for one that actually needs to be.

### PROBABLY NOT DOING THIS GridSearchCV for Decision Tree:

In [ ]:
"""param_grid = {
    'criterion': ['gini', 'entropy'],
    'max_depth': [None, 1, 5, 10, 20],
    'min_samples_split': [0.1, 0.2, 0.5, 0.75],
    'min_samples_leaf':[1,3,5,10]
}

gs_tree = GridSearchCV(DT_bal, param_grid, cv=3)
gs_tree.fit(X_train, y_train)
"""

In [ ]:
"""gstree_accuracy=gs_tree.best_score_ 
pred_gstree=gs_tree.predict(X_train)
print(f"Testing Accuracy: {gstree_accuracy:.2%}")
print("")
# Mean training score
#dt_gs_training_score = np.mean(gs_tree.cv_results_['mean_train_score'])
# Mean test score
#dt_gs_testing_score = gs_tree.score(X_test,y_test)
#print(f"Mean Training Score: {dt_gs_training_score :.2%}")
#print(f"Mean Test Score: {dt_gs_testing_score :.2%}")
print(f"Best Parameter Combination Found During Grid Search:{gs_tree.best_params_}")
"""

In [ ]:
###model_results(gs_tree)

In [ ]:
###resu_gstree=save_metrics(gs_tree, y_train, pred_gstree)

In [ ]:
###best_tree=DecisionTreeClassifier(criterion='gini', class_weight='balanced', max_depth=1, min_samples_split=0.1, min_samples_leaf=1, random_state=19)

In [ ]:
###best_tree.fit(X_train, y_train)

In [ ]:
###preds_best3 = best_tree.predict(X_train)

In [ ]:
###print("Results with best Decision Tree considering class imbalance:\n")
###model_results(best_tree)

In [ ]:
###resu_best3=save_metrics(best_tree, y_train, preds_best3)

In [ ]:
###resu_gstree

In [ ]:
###resu_best3

In [ ]:
###plot_roc([logreg, DT1, DT_bal, best_tree], y_train)

### Random Forest

In [ ]:
# Instantiate and fit a RandomForestClassifier
forest = RandomForestClassifier(random_state=19)
forest.fit(X_train, y_train)

In [ ]:
# Training accuracy score
forest.score(X_train, y_train)

In [ ]:
preds_for=forest.predict(X_train)

In [ ]:
# This should be the accuracy
#mean_rf_cv_score = np.mean(cross_val_score(forest, X_train, y_train, cv=3))

In [ ]:
#mean_rf_cv_score

In [ ]:
model_results(forest)

In [ ]:
plot_roc([logreg, DT1, DT_bal, forest], y_train)

In [ ]:
compare_roc(forest)

In [ ]:
#resu_forest=save_metrics(forest, y_train, preds_for)

### Random Forest class imbalance:
Now accounting for the class imbalance in Random Forest.

In [ ]:
forest_bal = RandomForestClassifier(random_state=19, class_weight='balanced')
forest_bal.fit(X_train, y_train)

In [ ]:
# Training accuracy score
forest_bal.score(X_train, y_train)

In [ ]:
preds_forbal=forest_bal.predict(X_train)

In [ ]:
model_results(forest_bal)

In [ ]:
plot_roc([logreg, DT1, DT_bal, forest, forest_bal], y_train)

In [ ]:
compare_roc(forest_bal)

In [ ]:
#resu_forbal=save_metrics(forest_bal, y_train, preds_forbal)

### GridSearchCV for Random Forest:

In [ ]:
# create the grid with parameters:

rf_param_grid = { 'n_estimators':[10,30,100],
                 'criterion': ['gini', 'entropy'],
                 'max_depth': [None, 6, 10, 30],
                 'min_samples_split':[5, 10],
                 'min_samples_leaf':[3, 6]
    
}

In [ ]:
""" code with which we ran the GridSearchCV
rf_grid_search = GridSearchCV(forest_bal, rf_param_grid,cv=3)
rf_grid_search.fit(X_train, y_train)"""

Since the model took a very long time to fit I am going to pickle it, to save it for a later rerun without it having to fit again.

In [ ]:
filename = 'CVforest.pkl'

In [ ]:
""" code with which we saved the model
# save the model to disk

joblib.dump(rf_grid_search, filename)
"""

In [ ]:
# some time later...

# load the model from disk
loaded_model = joblib.load(filename)

In [ ]:
forest_accuracy=loaded_model.best_score_ 
best_params=loaded_model.best_params_
print(f"Testing Accuracy: {forest_accuracy:.2%}")
print("")
print(f"Best Parameters:{best_params}")

In [ ]:
for_CV1 = RandomForestClassifier(random_state=19,class_weight='balanced','criterion':'entropy',
                                 'max_depth': 6, 'min_samples_leaf': 6, 'min_samples_split': 5, 'n_estimators': 10)

In [ ]:
for_CVl.fit(X_train, y_train)

In [ ]:
preds_forCV1=for_CV1.predict(X_train)

In [ ]:
#resu_forCV1=save_metrics(for_CV1, y_train, preds_forCV1)
model_results(for_CV1)


In [ ]:
plot_roc([logreg, DT1, DT_bal, forest, forest_bal, for_CV1], y_train)

In [ ]:
compare_roc(for_CV1)

In [ ]:
#model_results(loaded_model) I do not think this makes sense but lets hear from Eva

In [ ]:
rf_param_grid2 = { 'n_estimators':[20,30,40],
                 'criterion': ['gini'],
                 'max_depth':[None],
                 'min_samples_split':[3,5],
                 'min_samples_leaf':[5, 10, 20]
    
}

In [ ]:
"""Code with which we ran the second grid search
rf_grid_search2 = GridSearchCV(forest_bal, rf_param_grid2,cv=3)
rf_grid_search2.fit(X_train, y_train)"""

In [ ]:
filename2 = 'forGrid2.pkl'

In [ ]:
""" code with which we saved the model
# save the model to disk

joblib.dump(rf_grid_search2, filename2)
"""

In [ ]:
# some time later...

# load the model from disk
loaded_model2 = joblib.load(filename2)

In [ ]:
forest_accuracy=loaded_model2.best_score_ 
best_params=loaded_model2.best_params_

In [ ]:
print(f"Testing Accuracy: {forest_accuracy:.2%}")
print("")
print(f"Best Parameters:{best_params}")

In [ ]:
for_CV2 = RandomForestClassifier(random_state=19,class_weight='balanced','criterion': 'gini', 'max_depth': None, 
                                 'min_samples_leaf': 5, 'min_samples_split': 3, 'n_estimators': 30)

In [ ]:
for_CV2.fit(X_train, y_train)

In [ ]:
preds_forCV2=for_CV2.predict(X_train)

In [ ]:
model_results(for_CV2)

In [ ]:
plot_roc([logreg, DT1, DT_bal, forest, forest_bal, for_CV1, for_CV2], y_train)

In [ ]:
compare_roc(for_CV2)

In [ ]:
rf_param_grid3 = { 'n_estimators':[10, 15],
                 'criterion': ['entropy'],
                 'max_depth': [None,3,6],
                 'min_samples_split':[1,3,5],
                 'min_samples_leaf':[6, 10, 20]
    
}

In [ ]:
"""Code with which we ran the third gridsearch
rf_grid_search3 = GridSearchCV(forest_bal, rf_param_grid,cv=3)
rf_grid_search3.fit(X_train, y_train)"""

In [ ]:
filename3 = 'CVforest3.pkl'

In [ ]:
"""Code with which we saved the third gridserch
joblib.dump(rf_grid_search3, filename3)
"""

In [ ]:
# some time later...

# load the model from disk
loaded_model3 = joblib.load(filename3)

In [ ]:
forest_accuracy=loaded_model3.best_score_ 
best_params=loaded_model3.best_params_
print(f"Testing Accuracy: {forest_accuracy:.2%}")
print("")
print(f"Best Parameters:{best_params}")

In [ ]:
for_CV3 = RandomForestClassifier(random_state=19,class_weight='balanced','criterion': 'entropy', 'max_depth': 6,
                                 'min_samples_leaf': 6, 'min_samples_split': 5, 'n_estimators': 10)

In [ ]:
for_CV3.fit(X_train, y_train)


In [ ]:
preds_forCV3=for_CV3.predict(X_train)

In [ ]:
model_results(for_CV3)

In [ ]:
plot_roc([logreg, DT1, DT_bal, forest, forest_bal,for_CV1, for_CV2, for_CV3], y_train)

In [ ]:
compare_ROC(for_CV3)

### Best Forest

Then I will run a forest with those best parameters we just found:

In [ ]:
#best_forest = RandomForestClassifier(n_estimators= 30 , criterion= 'gini', max_depth= None, 
#                    min_samples_split=5, min_samples_leaf=6, random_state=19, class_weight='balanced')
#best_forest.fit(X_train, y_train)

In [ ]:
#ypreds_train_bestfor = best_forest.predict(X_train)

In [ ]:
#print("Results for best Random Forest:\n")

#print_metrics(y_train, ypreds_train_bestfor)

In [ ]:
#model_results(best_forest)

### Including class imbalance: THIS SHOULD NOT BE NECESSARY ALREADY INCLUDED ON TOP
Now we will also use the balancing of the classes since we saw before that helped improving the recall rate:

In [ ]:
#best_forest_bal = RandomForestClassifier(n_estimators= 30 , criterion= 'gini', max_depth= None, 
#min_samples_split= 5, min_samples_leaf=6, 
#                                     class_weight='balanced',random_state=19,)
#best_forest_bal.fit(X_train, y_train)

In [ ]:
#ypreds_train_bestfor_bal = best_forest_bal.predict(X_train)

In [ ]:
#print("Results for best Random Forest Balanced:\n")

#print_metrics(y_train, ypreds_train_bestfor_bal)

In [ ]:
#model_results(best_forest_bal)

### XGBooster

In [ ]:
# Instantiate XGBClassifier
xgb_model = xgb.XGBClassifier(random_state=19)

# Fit XGBClassifier
xgb_model.fit(X_train, y_train)

# Predict on training and test sets
xgb_preds = xgb_model.predict(X_train)
#test_preds = DT1.predict(X_test)

# Accuracy of training and test sets
#training_accuracy = accuracy_score(y_train, training_preds)
#test_accuracy = accuracy_score(y_test, test_preds)

#print('Training Accuracy: {:.4}%'.format(training_accuracy * 100))
#print('Validation Accuracy: {:.4}%'.format(test_accuracy * 100))

In [ ]:
print("Results for best Random Forest with XGBoost:\n")

print_metrics(y_train, xgb_preds)

In [ ]:
model_results(xgb_model)

In [ ]:
plot_roc([logreg, DT1, DT_bal, forest, forest_bal,for_CV1, for_CV2, for_CV3, xgb_model], y_train)

In [ ]:
compare_roc(xgb_model)

# To draw some conclusions:
Now in order to draw some conclusions let us look at the results of the different models together and we will plot the ROC curve for all the different models we saw.
<br>Once we can identify the best model we can see what the most important features were for that model.

In [ ]:
#models=[DT1, DT_bal, logreg, forest, best_forest, best_forest_bal, xgb_model]

In [ ]:
"""colors = sns.color_palette('Set1')
aucs=[]

plt.figure(figsize=(10,8))

for i in (models):
    
    yhat = i.predict_proba(X_train)
    yhat = yhat[:, 1]
    #fpr, tpr, thresholds= roc_curve(y_train, yhat)
    fpr, tpr, thresholds = roc_curve(y_train, yhat)
    #plot_roc_curve(DT1, y_test, yhat)
    # plot the roc curve for the model
    plt.plot(fpr, tpr, label=i)
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.legend()
    aucs.append(auc(fpr, tpr))
#    print('AUC for {}: {}'.format(i, auc(fpr, tpr)))
#    print('-------------------------------------------------------------------------------------')
#    lw = 2
#    plt.plot(fpr, tpr,
#             lw=lw, label='ROC curve {}'.format(i))

plt.plot([0, 1], [0, 1], color='navy', linestyle='--', label='No Skill')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])

plt.yticks([i/20.0 for i in range(21)])
plt.xticks([i/20.0 for i in range(21)])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic (ROC) Curve')
plt.legend(loc='lower right')
plt.show()"""

### I SHOULD CHANGE THIS SINCE I ALREADY SAVED THE RESULTS DOWN THE LINE

In [ ]:
"""precisions=[]
recalls=[]
accuracies=[]
F1s=[]
for i in (models):
    print(f"{Fore.BLUE} Model {Style.RESET_ALL}", i)
    preds=i.predict(X_train)
    precisions.append(precision_score(y_train, preds))
    recalls.append(recall_score(y_train, preds))
    accuracies.append(accuracy_score(y_train, preds))
    F1s.append(f1_score(y_train, preds))

"""

In [ ]:
"""### SORT THIS DF BY SOMETHING

results=pd.DataFrame([])
results['model']=models
results['precision']=precisions
results['recall']=recalls
results['accuracy']=accuracies
results['F1']=F1s
results['AUC']=aucs
results"""

In [ ]:
### Here I would compare just the 2/3 best confusion matrices
#for i in (models):
#    preds=i.predict(X_train)
#    print(f"{Fore.BLUE} Model {Style.RESET_ALL}", i)
#    plot_matrix(y_train, preds)

In [ ]:
"""predictions=[]
plt.figure(figsize=(15,15))
#    fig = plt.subplots(ncols=3, figsize=(15, 5))
for i in models_fi:
    preds=i.predict(X_train)
    predictions.append(preds)
    print(i)
    cm = confusion_matrix(y_train, preds)
    cmn = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    sns.heatmap(cmn, annot=True, fmt='.2f', cmap='BuPu')
    plt.ylabel('Actual')
    plt.xlabel('Predicted')
plt.show(block=False)"""

In [ ]:
models_fi=[logreg, DT1, DT_bal, forest, forest_bal,for_CV1, for_CV2, for_CV3, xgb_model]

In [ ]:
models_b=[DT_bal, forest_bal, xgb_model]

In [ ]:
"""import seaborn as sns
import matplotlib.pyplot as plt
sns.set_theme(style="whitegrid")

# Initialize the matplotlib figure
colors=['black', 'purple','blue', 'red','orange', 'yellow']
f, ax = plt.subplots(figsize=(12, 15))
for i, j in zip(models_b, colors):
    n_features = X_train.shape[1]
    plt.barh(range(n_features), (i.feature_importances_), alpha=0.7, color=j)
    plt.yticks(np.arange(n_features), X_train.columns.values)
    plt.legend()
#sns.barplot(x="total", y="abbrev", data=crashes,
#            label="Total", color="b")

# Plot the crashes where alcohol was involved
#sns.set_color_codes("muted")
#sns.barplot(x="alcohol", y="abbrev", data=crashes,
#            label="Alcohol-involved", color="b")

# Add a legend and informative axis label
    ax.legend (models_fi)
    ax.set(ylabel="features", xlabel="Feature Importance")"""

In [ ]:
##Here if I want I could create a df with all the feature importances. But maybe not necessary.
# Feature Importance
feat=[]
imp=[]
mod=[]
for i in models_b:
    feature_used = df.columns
    for fi, feature in zip(i.feature_importances_, feature_used):
        imp.append(fi)
        feat.append(feature)

In [ ]:
feat_imp=pd.DataFrame([])
feat_imp['model']=mod
feat_imp['features']=feat
feat_imp['importance']=imp

feat_imp.head()

In [ ]:
feat_imp['model'][0:13]='Tree Balanced'
feat_imp['model'][13:26]='Forest Balanced'
feat_imp['model'][26:]='XGB'

In [ ]:
plt.style.use('seaborn-poster')
#plt.figure(figsize=(30,15))
#features = sns.load_dataset("feat_imp")

# Draw a nested barplot by species and sex
g = sns.catplot(
    data=feat_imp, kind="bar",
    x="features", y="importance", hue="model",aspect=20/15
#    errorbar="sd", palette="dark", alpha=.6, height=6
)
g.set_axis_labels("", "Feature Importance")
g.fig.set_figwidth(20)
g.fig.set_figheight(10)
g.set_xticklabels(labels=feat, rotation=90);

In [ ]:
"""

Clearly from the df the results are too many. 
Better to make a graph but maybe only with the best 3 models or it's gonna get confusing.

"""

From what we can observe the five most important features in our selection overall for the models are:
    Pneumonia, Age, Diabetes, Renal Chronic, Hypertension and Sex?
    IDK I have to study this.

I should take the best model,
look at the list of Most Relevant Features,
and then study them.
For age, the only numeric one, I can divide it in bins and then plot for each bin the number of patients hospitalized.
To get a sense of what patients are more at risk.
For the other factors just say Drs can take care of those factors more, CDC could start campaigns and study more in depth people with those diseases.

### Code from Eva model Results

In [ ]:
def model_results(model):
    with plt.style.context('seaborn-talk'):
        fig, (ax1, ax2) = plt.subplots(ncols=2, figsize=(15, 5))
        # Create Confusion Matrix for the test set
        plot_confusion_matrix(model, X_train, y_train, normalize = 'true', ax=ax1,  cmap = 'Greens')
        ax1.grid(False)
        ax1.set_title("Train Confusion Matrix")
        # Create Roc curve for the test and train for TP and FP rates
        plot_roc_curve(model, X_train, y_train, ax=ax2, color='green', name ='Train ROC curve')
#        plot_roc_curve(model, X_test, y_test, ax=ax2, color = ‘blue’, name ='Test ROC curve' )
        ax2.plot([0, 1], [0, 1], color='black', lw=2, linestyle='-')
        ax2.set_xlabel('False Positive Rate')
        ax2.set_ylabel('True Positive Rate')
        ax2.set_title('Receiver Operating Characteristic (ROC) Curve')
        plt.show()
        #Create a classification report
        y_pred = model.predict(X_train)
        print(classification_report(y_train, y_pred))
        ######***NEW LINE*** Print CV ROC_AUC score, and roc_auc score for test/train
        #Print CV ROC_AUC score
        roc_score_train_cv = cross_val_score(estimator=model, X=X_train,  y=y_train,
                                        cv=StratifiedKFold(shuffle=True), scoring='roc_auc').mean()
        print(f"Mean Cross Validated Roc_Auc Score: {roc_score_train_cv :.0%}")
        #print roc_auc for test and train
        roc_score_train = roc_auc_score(y_train, model.predict_proba(X_train)[:, 1])
        print(f"Train Roc_Auc Score: {roc_score_train :.0%}")
#        roc_score_test = roc_auc_score(y_test, model.predict_proba(X_test)[:, 1])
#        print(f”Test Roc_Auc Score: {roc_score_test :.0%}“)
        accuracy_train = model.score(X_train, y_train)
#        accuracy_test = model.score(X_test, y_test)
        print(f"Train Accuracy Score: {accuracy_train :.0%}")
#        print(f”Test Accuracy Score: {accuracy_test :.0%}“)